# Assistants

[Assistants](https://langchain-ai.github.io/langgraph/concepts/assistants/#resources) 为开发者提供了一种快速简便的方法来修改和版本化 agents，以便进行实验。

## 为 Graph 提供配置

我们的 `task_maistro` graph 已经设置好使用 assistants 了！

它定义了一个 `configuration.py` 文件并在 graph 中加载。

我们在 graph 节点内部访问可配置字段（`user_id`、`todo_category`、`task_maistro_role`）。

## 创建 Assistants

那么，对于我们一直在构建的 `task_maistro` 应用，assistants 的实际使用场景是什么呢？

对我来说，就是能够为不同类别的任务拥有独立的 ToDo 列表。

例如，我希望一个 assistant 用于个人任务，另一个用于工作任务。

这些都可以通过 `todo_category` 和 `task_maistro_role` 可配置字段轻松配置。

![Screenshot 2024-11-18 at 9.35.55 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/673d50597f4e9eae9abf4869_Screenshot%202024-11-19%20at%206.57.01%E2%80%AFPM.png)

In [ ]:
%%capture --no-stderr
# ================== 安装 LangGraph SDK ==================

# LangGraph SDK 知识点:
# - LangGraph SDK 是用于与 LangGraph Platform 部署交互的 Python 客户端
# - 提供以下核心功能:
#   1. client.assistants: 管理 assistants (创建、更新、搜索、删除)
#   2. client.threads: 管理对话线程
#   3. client.runs: 执行 graph runs 并获取结果
#   4. client.store: 访问持久化存储
#
# 与 LangGraph CLI 的区别:
# - CLI: 用于构建和部署 (langgraph build, langgraph up)
# - SDK: 用于与运行中的 deployment 交互
#
# 使用场景:
# - 从 Python 代码中调用已部署的 graphs
# - 管理 assistants 和 configurations
# - 访问对话历史和状态
# - 构建前端应用与 LangGraph Platform 集成
#
# 安装说明:
# - -U flag: 升级到最新版本
# - langgraph_sdk: SDK 包名称

%pip install -U langgraph_sdk

这是我们部署 graph 时创建的默认 assistant。

In [ ]:
# ================== 连接到 LangGraph Platform Deployment ==================

from langgraph_sdk import get_client

# LangGraph SDK 知识点:
# - get_client(): 创建 LangGraph Platform client
# - client 是与 deployment 交互的主要接口
# - 需要提供 deployment 的 URL
#
# 本地 deployment 的 URL:
# - CLI deployment (docker compose): http://localhost:8123
# - LangGraph Studio: http://localhost:2024
#
# 远程 deployment 的 URL:
# - LangGraph Cloud: https://<deployment-id>.langchain.com
# - 自托管: https://your-domain.com

url_for_cli_deployment = "http://localhost:8123"

# 创建 client 实例
# - 这个 client 可以用于:
#   1. 管理 assistants (client.assistants)
#   2. 创建和管理 threads (client.threads)
#   3. 执行 graph runs (client.runs)
#   4. 访问 store (client.store)
client = get_client(url=url_for_cli_deployment)

### Personal assistant

这是我将用于管理个人任务的 personal assistant。

In [ ]:
# ================== 创建 Personal Assistant ==================

# Assistant 知识点:
# - Assistant 是 graph 的配置化实例
# - 每个 assistant 有唯一的 assistant_id
# - 可以为同一个 graph 创建多个 assistants，使用不同配置
# - Assistants 存储在 PostgreSQL 中，可以跨会话持久化
#
# 为什么使用 Assistants:
# - 为不同用例创建不同配置 (personal vs work)
# - 版本化 agent 行为和配置
# - 快速实验不同的系统提示词和参数
# - 为不同用户或团队创建隔离的环境

# Python 知识点:
# - await: 异步等待函数完成
# - LangGraph SDK 使用异步 I/O 以提高性能

personal_assistant = await client.assistants.create(
    # 第一个参数: graph_id
    # - "task_maistro" 是我们在 module 5 中创建并部署的 graph
    # - 这个名称来自 deployment 配置 (langgraph.json)
    "task_maistro",
    
    # config 参数: 可配置字段
    # - 这些字段在 configuration.py 中定义
    # - 会传递给 graph 节点使用
    config={"configurable": {"todo_category": "personal"}}
    
    # 配置说明:
    # - todo_category="personal": 这个 assistant 管理个人任务
    # - 与 "work" category 分开，实现任务隔离
)

# 打印 assistant 信息
# - assistant_id: 唯一标识符
# - graph_id: 关联的 graph
# - config: 配置信息
# - version: assistant 版本号（初始为 1）
print(personal_assistant)

让我们更新这个 assistant，为了方便起见添加我的 `user_id`，[创建它的新版本](https://langchain-ai.github.io/langgraph/cloud/how-tos/assistant_versioning/#create-a-new-version-for-your-assistant)。

In [ ]:
# ================== 定义 Personal Assistant 的系统提示词 ==================

# Prompt Engineering 知识点:
# - 系统提示词定义 assistant 的行为和个性
# - 清晰的指令可以引导 model 按预期方式工作
# - 包含具体示例和格式要求

task_maistro_role = """You are a friendly and organized personal task assistant. Your main focus is helping users stay on top of their personal tasks and commitments. Specifically:

- Help track and organize personal tasks
- When providing a 'todo summary':
  1. List all current tasks grouped by deadline (overdue, today, this week, future)
  2. Highlight any tasks missing deadlines and gently encourage adding them
  3. Note any tasks that seem important but lack time estimates
- Proactively ask for deadlines when new tasks are added without them
- Maintain a supportive tone while helping the user stay accountable
- Help prioritize tasks based on deadlines and importance

Your communication style should be encouraging and helpful, never judgmental. 

When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Would you like to add one to help us track it better?"""

# Personal Assistant 系统提示词设计要点:
# - 友好和支持性的语气 ("friendly and organized", "encouraging")
# - 主动提供帮助 ("Proactively ask for deadlines")
# - 按截止日期分组任务 (overdue, today, this week, future)
# - 温和地鼓励而非批评 ("never judgmental", "gently encourage")
# - 提供具体的回应模板 (示例对话)

# ========== 组合完整的 Assistant 配置 ==========

configurations = {
    # todo_category: 任务类别
    # - "personal": 个人任务 (家庭、个人项目等)
    # - 与 "work" 分开存储
    "todo_category": "personal",
    
    # user_id: 用户标识符
    # - 用于在 Store 中区分不同用户的数据
    # - 对应 module 5 中的 namespace: ("todo", user_id)
    "user_id": "lance",
    
    # task_maistro_role: agent 的系统提示词
    # - 定义 agent 的行为和个性
    # - 这是 Personal Assistant 特有的提示词
    "task_maistro_role": task_maistro_role
}

# ================== 更新 Assistant ==================

# Assistant 版本化知识点:
# - 每次更新 assistant 都会创建新版本
# - 版本号自动递增 (1 -> 2 -> 3...)
# - 旧版本仍然保存在数据库中
# - 可以回滚到之前的版本
#
# 为什么需要版本化:
# - 实验不同的配置和提示词
# - 回滚到之前工作正常的版本
# - 追踪配置变更历史
# - A/B 测试不同的 agent 行为

personal_assistant = await client.assistants.update(
    # 第一个参数: assistant_id
    # - 要更新的 assistant 的唯一标识符
    # - 从创建时返回的 assistant 对象中获取
    personal_assistant["assistant_id"],
    
    # config 参数: 新的配置
    # - 完全替换之前的配置
    # - 这里添加了 user_id 和 task_maistro_role
    config={"configurable": configurations}
)

# 打印更新后的 assistant
# - 注意 version 字段从 1 变为 2
# - config 现在包含所有三个字段
# - created_at 和 updated_at 时间戳不同
print(personal_assistant)

# 输出示例:
# {
#   'assistant_id': 'e6ab9c39-4b56-4db9-bb39-a71484c5d408',
#   'graph_id': 'task_maistro',
#   'version': 2,  # ← 版本号递增
#   'config': {
#     'configurable': {
#       'user_id': 'lance',
#       'todo_category': 'personal',
#       'task_maistro_role': '...'  # ← 新增字段
#     }
#   }
# }

### Work assistant

现在，让我们创建一个 work assistant。我将用它来管理我的工作任务。

In [ ]:
# ================== 定义 Work Assistant 的系统提示词 ==================

# Work Assistant vs Personal Assistant 对比:
# - Work: 更注重效率、现实时间框架、团队依赖
# - Personal: 更注重支持性、鼓励、个人责任
#
# Prompt Engineering 差异化要点:
# - "focused and efficient" vs "friendly and organized"
# - 提供具体的任务类型时间估算
# - "professional tone" vs "encouraging and helpful"
# - 考虑团队依赖关系

task_maistro_role = """You are a focused and efficient work task assistant. 

Your main focus is helping users manage their work commitments with realistic timeframes. 

Specifically:

- Help track and organize work tasks
- When providing a 'todo summary':
  1. List all current tasks grouped by deadline (overdue, today, this week, future)
  2. Highlight any tasks missing deadlines and gently encourage adding them
  3. Note any tasks that seem important but lack time estimates
- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:
  • Developer Relations features: typically 1 day
  • Course lesson reviews/feedback: typically 2 days
  • Documentation sprints: typically 3 days
- Help prioritize tasks based on deadlines and team dependencies
- Maintain a professional tone while helping the user stay accountable

Your communication style should be supportive but practical. 

When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Based on similar tasks, this might take [suggested timeframe]. Would you like to set a deadline with this in mind?"""

# Work Assistant 系统提示词设计要点:
# - 实用和高效的方法 ("focused", "realistic timeframes")
# - 基于任务类型的具体时间建议 (1-3 days)
# - 考虑团队依赖 ("team dependencies")
# - 专业但支持性的语气 ("professional", "supportive but practical")
# - 提供数据驱动的建议 ("Based on similar tasks")

# ========== 组合 Work Assistant 配置 ==========

configurations = {
    # todo_category: "work"
    # - 工作任务 (项目、会议、开发任务等)
    # - 与 "personal" 完全隔离
    "todo_category": "work",
    
    # user_id: 相同的用户
    # - 同一个用户可以有多个 assistants
    # - 通过 todo_category 区分
    "user_id": "lance",
    
    # task_maistro_role: Work Assistant 专用提示词
    # - 与 Personal Assistant 的提示词完全不同
    # - 针对工作场景优化
    "task_maistro_role": task_maistro_role
}

# ================== 创建 Work Assistant ==================

# 注意: 这里使用 create() 而非 update()
# - 这是一个全新的 assistant
# - 与 Personal Assistant 并存
# - 有独立的 assistant_id
# - 版本号从 1 开始

work_assistant = await client.assistants.create(
    # 相同的 graph_id
    # - Personal 和 Work assistants 使用同一个 graph
    # - 通过配置实现不同的行为
    "task_maistro",
    
    # Work-specific 配置
    config={"configurable": configurations}
)

# 打印 Work Assistant 信息
# - 新的 assistant_id (与 Personal 不同)
# - version: 1 (新 assistant)
# - config 包含 work-specific 配置
print(work_assistant)

# 多 Assistant 架构的优势:
# - 任务隔离: personal 和 work todos 分开
# - 行为差异化: 不同的系统提示词和优先级逻辑
# - 灵活扩展: 可以轻松添加更多类别 (家庭、项目等)
# - 用户体验: 用户可以在不同 context 之间切换

## 使用 Assistants

Assistants 将保存到我们 deployment 中的 `Postgres` 数据库。

这使我们可以使用 SDK 轻松[搜索](https://langchain-ai.github.io/langgraph/cloud/how-tos/configuration_cloud/) assistants。

In [ ]:
# ================== 搜索所有 Assistants ==================

# client.assistants.search() 知识点:
# - 返回所有 assistants 的列表
# - Assistants 存储在 PostgreSQL 数据库中
# - 可以跨会话持久化
# - 支持按条件过滤（这里返回所有）

assistants = await client.assistants.search()

# 遍历并打印关键信息
for assistant in assistants:
    print({
        'assistant_id': assistant['assistant_id'],  # 唯一标识符
        'version': assistant['version'],  # 版本号
        'config': assistant['config']  # 配置信息（包含 user_id, category, role）
    })

# 输出示例:
# - 可能看到多个 assistants
# - Personal assistant (todo_category='personal')
# - Work assistant (todo_category='work')
# - 可能还有之前创建的其他 assistants

我们可以使用 SDK 轻松管理它们。例如，我们可以删除不再使用的 assistants。
> 视频中的语法略有偏差。下面更新的代码创建了一个临时 assistant 然后删除它。

In [ ]:
# ================== 演示删除 Assistant ==================

# 创建临时 assistant 用于演示
temp_assistant = await client.assistants.create(
    "task_maistro",
    config={"configurable": configurations}
)

# 删除前: 显示所有 assistants
assistants = await client.assistants.search()
for assistant in assistants:
    print(f"before delete: {{'assistant_id': {assistant['assistant_id']}}}")

# ========== 删除 Assistant ==========

# client.assistants.delete() 知识点:
# - 永久删除 assistant 及其所有版本
# - 不会删除使用该 assistant 创建的 threads
# - 不会删除 threads 中的对话历史
# - 只是删除 assistant 配置本身

await client.assistants.delete(assistants[-1]["assistant_id"])  # 删除最后一个（临时）
print()

# 删除后: 显示剩余 assistants
assistants = await client.assistants.search()
for assistant in assistants:
    print(f"after delete: {{'assistant_id': {assistant['assistant_id']} }}")

# Assistant 管理最佳实践:
# - 定期清理不再使用的 assistants
# - 使用描述性的 name 字段（这里未使用）
# - 保持 assistant 数量可管理
# - 考虑使用命名约定区分不同类型

让我们设置 `personal` 和 `work` assistants 的 assistant IDs，我将使用它们。

In [ ]:
# ================== 设置要使用的 Assistant IDs ==================

# 从搜索结果中提取 assistant IDs
# - assistants[0]: Work assistant
# - assistants[1]: Personal assistant
#
# 注意: 索引可能因创建顺序而异
# 最佳实践: 通过 config 匹配而非索引

work_assistant_id = assistants[0]['assistant_id']
personal_assistant_id = assistants[1]['assistant_id']

# 这些 IDs 将在后续 cells 中用于:
# - 创建 threads
# - 执行 runs
# - 与特定 assistant 交互

### Work assistant

让我们为 work assistant 添加一些 ToDos。

In [ ]:
# ================== 与 Work Assistant 交互 - 创建 ToDos ==================

from langchain_core.messages import HumanMessage
from langchain_core.messages import convert_to_messages

# 用户输入: 创建或更新几个工作任务
user_input = "Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday."

# ========== 创建新 Thread ==========
# Thread 知识点:
# - Thread 代表一个对话会话
# - 每个 thread 有独立的对话历史
# - 存储在 PostgreSQL 中
# - 与 assistant 关联使用
thread = await client.threads.create()

# ========== 执行 Run 并流式输出 ==========
# client.runs.stream() 知识点:
# - 异步流式执行 graph
# - 实时获取中间结果
# - stream_mode="values": 返回完整状态
# - 每个 chunk 包含当前所有 messages

async for chunk in client.runs.stream(
    thread["thread_id"],  # 对话线程 ID
    work_assistant_id,  # 使用 Work Assistant
    input={"messages": [HumanMessage(content=user_input)]},  # 用户消息
    stream_mode="values"  # 流式模式
):
    if chunk.event == 'values':
        state = chunk.data
        # 打印最新消息
        # - Human: 用户输入
        # - AI: agent 响应
        # - Tool: tool 执行结果
        convert_to_messages(state["messages"])[-1].pretty_print()

# 预期行为:
# - Work Assistant 分析任务
# - 调用 UpdateMemory tool (update_type='todo')
# - 根据系统提示词提供专业的响应
# - 可能建议时间估算（基于任务类型）

In [ ]:
# ================== 继续对话 - 创建另一个 ToDo ==================

# 用户输入: 创建另一个任务
user_input = "Create another ToDo: Finalize set of report generation tutorials."

# 注意: 使用新的 thread
# - 每次创建新 thread 代表新对话
# - 不会保留之前 thread 的对话历史
# - 但 assistant 配置保持不变
# - ToDo 数据通过 Store 跨 threads 持久化
thread = await client.threads.create()

async for chunk in client.runs.stream(
    thread["thread_id"],
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values"
):
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

# Work Assistant 特点:
# - 注意它如何根据系统提示词行为
# - 可能提供基于任务类型的时间估算
# - 专业但支持性的语气

Assistant 使用它的 instructions 来引导任务创建！

它要求我指定截止日期 :)

In [ ]:
# ================== 在同一 Thread 中更新 ToDo ==================

# 用户输入: 为任务设置截止日期
user_input = "OK, for this task let's get it done by next Tuesday."

# 关键区别: 使用同一个 thread
# - 保留对话上下文
# - Assistant 知道"this task"指的是哪个任务
# - 这是 thread 的价值所在: 上下文感知

async for chunk in client.runs.stream(
    thread["thread_id"],  # ← 相同的 thread_id
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values"
):
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

# Thread vs Store 的区别:
# - Thread: 对话历史（短期记忆）
#   - 只在当前会话中有效
#   - 用于理解上下文引用（"this task", "that one"）
#
# - Store: 数据持久化（长期记忆）
#   - 跨所有 threads 共享
#   - 存储 todos, profile 等数据
#   - 即使在新 thread 中也可访问

### Personal assistant

同样，我们可以为 personal assistant 添加 ToDos。

In [ ]:
# ================== 与 Personal Assistant 交互 ==================

# 用户输入: 创建个人任务
user_input = "Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points."

# 创建新 thread 用于 Personal Assistant
thread = await client.threads.create()

async for chunk in client.runs.stream(
    thread["thread_id"],
    personal_assistant_id,  # ← 使用 Personal Assistant
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values"
):
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

# Personal vs Work Assistant 行为对比:
# - Personal: 友好、鼓励、温和提醒
#   - "Would you like to add a deadline?"
#   - 支持性语气
#
# - Work: 专业、实用、基于数据的建议
#   - "Based on similar tasks, this might take..."
#   - 提供具体时间估算
#
# 相同点:
# - 都管理 ToDos
# - 都使用相同的 graph (task_maistro)
# - 都访问相同的 Store 结构
#
# 不同点:
# - todo_category 不同 (personal vs work)
# - 系统提示词完全不同
# - 行为和语气差异化

In [ ]:
# ================== 获取 ToDo Summary ==================

# 用户输入: 请求任务摘要
user_input = "Give me a todo summary."

# 创建新 thread
# - 这个 thread 不包含之前的对话历史
# - 但 Personal Assistant 仍然可以访问所有 personal todos
# - 因为 todos 存储在 Store 中（跨 threads）
thread = await client.threads.create()

async for chunk in client.runs.stream(
    thread["thread_id"],
    personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values"
):
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

# Assistant 的 ToDo Summary 功能:
# - 按截止日期分组 (overdue, today, this week, future)
# - 高亮缺少截止日期的任务
# - 注意缺少时间估算的任务
# - 根据系统提示词调整语气和建议
#
# 这个功能在两个 assistants 中:
# - Personal: 温和鼓励添加截止日期
# - Work: 提供基于历史的时间估算建议
#
# Assistants 的价值总结:
# - 同一个 graph，不同的配置和行为
# - 版本化和实验友好
# - 轻松实现多用户、多场景支持
# - 通过 config 实现灵活的个性化